# Custom strategy registry example

This example demos the concept of ReasoningNodes, Strategies, and StrategyRegistries in CoT-Forge. Users can create their own strategies and use them within search algorithms, or even write their own search algorithms.

#### Imports

In [19]:
from dataclasses import dataclass

from cot_forge import (
  CoTBuilder,
  InitializeCoT,
  LLMJudgeVerifier,
  NaiveLinearSearch,
  Strategy,
  StrategyRegistry,
  default_strategy_registry,
)
from cot_forge.llm import LMStudioProvider

#### Strategy registries

In CoT-Forge, each chain is made up of connected [ReasoningNodes](../src/cot_forge/reasoning/types.py#L21). Each node has an associated [Strategy](../src/cot_forge/reasoning/strategies/strategies.py#L63) that prompts the model on how to continue the reasoning process.

Strategy registries contain a collection of reasoning strategies and are passed to search algorithms to sample from. Let's take a look at the default strategy registry.

In [2]:
default_strategy_registry

StrategyRegistry(strategies=[initialize, backtrack, explore_new_paths, correction, validation])

We see that the strategy registry contains a list of five reasoning strategies. We can access individual Strategy data classes with the `get_strategy` method.

In [7]:
backtrack_strategy = default_strategy_registry.get_strategy('backtrack')
backtrack_strategy.get_metadata()

{'name': 'backtrack',
 'description': 'Backtrack: Revise the reasoning by **backtracking to an earlier point** in your analysis. Identify a specific earlier step where the reasoning began to go off track, return to that point, and develop a new chain of thought from there rather than continuing from your latest conclusion.',
 'is_initial': False,
 'minimum_depth': 2}

Above we see the data in the backtrack strategy.
- The `description` is what is used in the prompt to the model. 
- `is_initial` indicates whether this strategy kicks off the reasoning chain.
- `minimum_depth` is the minimum number of reasoning nodes that must be present in the chain before this strategy can be used.

 #### Creating a custom strategy registry

In [ ]:
# We'll create our own strategy registry, starting with just the InitializeCoT strategy
my_registry = StrategyRegistry(strategies=[InitializeCoT])

# We can define our own strategy by creating a new dataclass that inherits from Strategy
@dataclass(frozen=True)
class Analogy(Strategy):
  name = "analogy"
  description = "Analogical Reasoning: Enhance your reasoning by using **analogies or metaphors** to reinterpret challenging aspects of your analysis. Draw parallels to more familiar concepts that can illuminate the current problem from a different angle."
  is_initial = False

# Rgister the new strategy with the registry
my_registry.register(Analogy)

# Alternatively, we can create and register a strategy in one step using the create_and_registr method
my_registry.create_and_register(
  name="refutation",
  description="Refutation: Strengthen your reasoning by **actively seeking counterarguments** or alternative perspectives. This strategy encourages you to critically evaluate your own conclusions and consider potential weaknesses in your analysis.",
  is_initial=False,
)

# List our strategies to make sure they are registered
my_registry.list_strategies()

['initialize', 'analogy', 'refutation']

#### Creating and running a CoTBuilder.
Other examples show how to create and run a CoTBuilder in more detail. Here we will just create a simple CoTBuilder, using NaiveLinearSearch as the search algorithm and run it with the custom strategy registry.

In [23]:
question = """Four people - Alex, Blake, Casey, and Dana - are suspects in a theft. When questioned, they make the following statements:

Alex says: "Blake did it."
Blake says: "Casey did it."
Casey says: "Blake is lying."
Dana says: "I didn't do it."
The detective knows that exactly three of these statements are true and one is false. Who committed the theft?"""

ground_truth = "Blake"

In [34]:
llama = LMStudioProvider(model_name="meta-llama-3-8b-instruct")
builder = CoTBuilder(
    search_llm=llama,
    post_processing_llm=llama, 
    search=NaiveLinearSearch(max_depth=3),
    verifier=LLMJudgeVerifier(llama, strict=False),
    strategy_registry=my_registry,
)

In [38]:
search_result, reasoning = builder.process(
  question=question,
  llm_kwargs={"temperature": 0.6},
  ground_truth_answer=ground_truth, 
  only_successful=False
)

#### Examining results

In [39]:
print(f"Search Result: {search_result}")
terminal_node = search_result.terminal_nodes[0]
for i, node in enumerate(terminal_node.get_full_node_chain()):
  print(f"Step {i}: {node}")

Search Result: SearchResult(success=False, question=Four people - Alex, Blake, Cas..., num_terminal_nodes=1, num_successful_nodes=0, successful_answers=[])
Step 0: ReasoningNode(strategy=initialize, success=False, final=False, cot_steps=6)
Step 1: ReasoningNode(strategy=analogy, success=False, final=False, cot_steps=6)
Step 2: ReasoningNode(strategy=analogy, success=False, final=False, cot_steps=6)
Step 3: ReasoningNode(strategy=refutation, success=False, final=True, cot_steps=5)


We see above that the search was a failure, and the reasoning node steps were: initialize -> analogy -> analogy -> refutation. Let's examine the reasoning node steps in more detail.

In [47]:
terminal_node.get_full_node_chain()[2].cot

[{'action': 'Review',
  'content': 'The previous reasoning concluded that Dana committed the theft, but this conclusion was later found to be false.'},
 {'action': 'Inner Thinking',
  'title': 'Analogical Reasoning: Theft as a Puzzle',
  'content': "Let's consider the statements like pieces of a puzzle. We need to find the correct combination where three statements are true and one is false. This analogy can help us identify patterns and connections between the statements."},
 {'action': 'Inner Thinking',
  'title': 'Re-examining Accusations through Analogies',
  'content': "Alex accuses Blake, and Blake accuses Casey. This is similar to a game of 'whisper down the lane' where information gets distorted. We need to consider the potential for misinformation or misdirection."},
 {'action': 'Inner Thinking',
  'title': "Dana's Denial: A Red Herring?",
  'content': "Dana's statement 'I didn't do it' is a straightforward denial, but what if it's a red herring? What if Dana is trying to dist

In [50]:
terminal_node.get_full_node_chain()[3].cot

[{'action': 'Review',
  'content': 'The previous reasoning concluded that Alex committed the theft, but this conclusion was later found to be false.'},
 {'action': 'Inner Thinking',
  'title': 'Re-examining the Statements',
  'content': "Let's re-analyze the statements with a critical eye. Dana's denial could still be true or false. If she's telling the truth, then one of the other three must be lying. But if she's lying, then her statement would contradict Alex's accusation of Blake."},
 {'action': 'Inner Thinking',
  'title': "Casey's Statement: A Clue?",
  'content': 'Casey says Blake is lying. This statement could be true or false. If Casey is telling the truth, then Blake did not commit the theft. But if Casey is lying, then Blake might actually be innocent.'},
 {'action': 'Inner Thinking',
  'title': "Alex's Accusation: A Red Herring?",
  'content': "What if Alex's accusation of Blake doing it is a red herring? What if Alex is trying to distract us from the real culprit?"},
 {'ac

We se that node 2 correctly tried to use analogies by thinking of the problem as a puzzle, while node 3 tries to re-examine and refute the previous steps. However, our reasoning strategies do not seem to be a good fit for this puzzle, and the search failed.